In [10]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen

In [5]:
ticker = 'ZM'

In [28]:
#scapper - needs user:agent 

def get_institutional_ownership(ticker):
    values = []
    columns = ['Institutional_Ownership', 'Change_Instituional_Ownership_3months']
    url = ("http://finviz.com/quote.ashx?t=" + ticker.lower())
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = bs(webpage, "html.parser")
    inst_ownership = soup.find(text = 'Inst Own').find_next(class_ = 'snapshot-td2').text
    values.append(inst_ownership)
    change_inst_ownership_3mon = soup.find(text = 'Inst Trans').find_next(class_ = 'snapshot-td2').text
    values.append(change_inst_ownership_3mon)
    df = pd.DataFrame(data = values, index = columns)
    return df

In [29]:
get_institutional_ownership('ZM')

,0
Institutional_Ownership,57.50%
Change_Instituional_Ownership_3months,0.65%


In [ ]:
def fundamental_metric(soup, metric):
    return soup.find(text = metric).find_next(class_='snapshot-td2').text

In [7]:
def get_fundamental_data(df):
    for symbol in df.index:
        try:
            url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
            soup = bs(requests.get(url).content) 
            for m in df.columns:                
                df.loc[symbol,m] = fundamental_metric(soup,m)                
        except Exception as e:
            print (symbol, 'not found')
    return df

In [8]:
stock_list = ['ZM']

metric = ['Inst Own',
'Inst Trans']

In [12]:
df = pd.DataFrame(index=stock_list,columns=metric)
df = get_fundamental_data(df)
df.astype(float64)

ZM not found


NameError: name 'float64' is not defined

In [14]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen

pd.set_option('display.max_colwidth', 25)

# Input
symbol = input('Enter a ticker: ')
print ('Getting data for ' + symbol + '...\n')

# Set up scraper
url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

def get_fundamentals():
    try:
        # Find fundamentals table
        fundamentals = pd.read_html(str(html), attrs = {'class': 'snapshot-table2'})[0]
        
        # Clean up fundamentals dataframe
        fundamentals.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
        colOne = []
        colLength = len(fundamentals)
        for k in np.arange(0, colLength, 2):
            colOne.append(fundamentals[f'{k}'])
        attrs = pd.concat(colOne, ignore_index=True)
    
        colTwo = []
        colLength = len(fundamentals)
        for k in np.arange(1, colLength, 2):
            colTwo.append(fundamentals[f'{k}'])
        vals = pd.concat(colTwo, ignore_index=True)
        
        fundamentals = pd.DataFrame()
        fundamentals['Attributes'] = attrs
        fundamentals['Values'] = vals
        fundamentals = fundamentals.set_index('Attributes')
        return fundamentals

    except Exception as e:
        return e

print ('Fundamental Ratios: ')
print(get_fundamentals())


Enter a ticker:  ZM


Getting data for ZM...

Fundamental Ratios: 
                 Values
Attributes             
Index                 -
Market Cap      107.74B
Income          425.50M
Sales             1.96B
Book/sh            5.27
...                 ...
ATR               18.83
Volatility  6.45% 4.72%
Prev Close       383.24
Price            384.53
Change            0.34%

[72 rows x 1 columns]
